# Create Kafka Topic

**Error Functions**

In [ ]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

**Set Admin Parameters**

In [ ]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
from confluent_kafka.admin import AdminClient, NewTopic
import json
import config               #hidden config file with keys and login credentials

#KAFKA variables
confluentClusterName = config.confluentClusterName
confluentBootstrapServers = config.confluentBootstrapServers
confluentTopicName = config.confluentTopicName
schemaRegistryUrl = config.schemaRegistryUrl
confluentApiKey = config.confluentApiKey
confluentSecret = config.confluentSecret
confluentRegistryApiKey = config.confluentRegistryApiKey
confluentRegistrySecret = config.confluentRegistrySecret

admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

**Create Topic**

In [ ]:
topic_list = []

topic_list.append(NewTopic("diabetesfinal-10", 1, 3))
admin_client.create_topics(topic_list)
futures = admin_client.create_topics(topic_list)

try:
    record_metadata = []
    for k, future in futures.items():
        # f = i.get(timeout=10)
        print(f"type(k): {type(k)}")
        print(f"type(v): {type(future)}")
        print(future.result())

except KafkaError:
    # Decide what to do if produce request failed...
    print(traceback.format_exc())
    result = 'Fail'
finally:
    print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
finally

--------------------------------------------------------------------------- 
 KafkaException Traceback (most recent call last)
 <command-1059330286235166> in <module> 
 11 print ( f"type(k): {type(k)}" ) 
 12 print ( f"type(v): {type(future)}" ) 
 ---> 13 print ( future . result ( ) ) 
 14 
 15 except KafkaError : 

 /usr/lib/python3.8/concurrent/futures/_base.py in result (self, timeout) 
 442 raise CancelledError ( ) 
 443 elif self . _state == FINISHED : 
 --> 444 return self . __get_result ( ) 
 445 else : 
 446 raise TimeoutError ( ) 

 /usr/lib/python3.8/concurrent/futures/_base.py in __get_result (self) 
 387 if self . _exception : 
 388 try : 
 --> 389 raise self . _exception
 390 finally : 
 391 # Break a reference cycle with the exception in self._exception 

 KafkaException : KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'diabetesfinal-10' already exists."}